This notebook verifies math in Appendix A. Perspective effect in Oh & Evans 2020.

In [2]:
from sympy import symbols, simplify, latex
from sympy import cos, sin, Matrix, diff, N
import numpy as np

ra, dec = symbols('alpha, delta')
vra,vdec,vr = symbols(r'v_\alpha, v_\delta, v_r')
vx,vy,vz = symbols('v_x v_y v_z')
delta_ra, delta_dec= symbols(r'\Delta\alpha \Delta\delta')

R = Matrix([
    [-sin(ra), cos(ra), 0.],
    [-sin(dec)*cos(ra), -sin(dec)*sin(ra), cos(dec)],
    [cos(dec)*cos(ra), cos(dec)*sin(ra), sin(dec)]
])

In [3]:
R

Matrix([
[           -sin(alpha),             cos(alpha),        0.0],
[-sin(delta)*cos(alpha), -sin(alpha)*sin(delta), cos(delta)],
[ cos(alpha)*cos(delta),  sin(alpha)*cos(delta), sin(delta)]])

In [4]:
simplify(R.inv()) == R.T

True

In [5]:
diff(R, ra)

Matrix([
[           -cos(alpha),            -sin(alpha), 0],
[ sin(alpha)*sin(delta), -sin(delta)*cos(alpha), 0],
[-sin(alpha)*cos(delta),  cos(alpha)*cos(delta), 0]])

In [6]:
diff(R, dec)

Matrix([
[                     0,                      0,           0],
[-cos(alpha)*cos(delta), -sin(alpha)*cos(delta), -sin(delta)],
[-sin(delta)*cos(alpha), -sin(alpha)*sin(delta),  cos(delta)]])

## Geneneral $\Delta v_\mathrm{sphere}$ to the first order

In [7]:
vvec = Matrix([
    [vx],
    [vy],
    [vz]
])

In [8]:
delta_v_sphere = diff(R, ra)*vvec*delta_ra + diff(R, dec)*vvec*delta_dec
delta_v_sphere

Matrix([
[                                                                                                               \Delta\alpha*(-v_x*cos(alpha) - v_y*sin(alpha))],
[ \Delta\alpha*(v_x*sin(alpha)*sin(delta) - v_y*sin(delta)*cos(alpha)) + \Delta\delta*(-v_x*cos(alpha)*cos(delta) - v_y*sin(alpha)*cos(delta) - v_z*sin(delta))],
[\Delta\alpha*(-v_x*sin(alpha)*cos(delta) + v_y*cos(alpha)*cos(delta)) + \Delta\delta*(-v_x*sin(delta)*cos(alpha) - v_y*sin(alpha)*sin(delta) + v_z*cos(delta))]])

We can express this with $v_\mathrm{sphere} = [v_\alpha,\,v_\delta,\,v_r]^T$ **at** $(\alpha,\,\delta)$.
Such first-order correction has been applied in e.g., Kuhn et al. 2019.
The limits of this is:
1. the mean velocity is estimaten in the projected space, where the perspective effect is **baked in** already
2. it is correct to only first-order in $\Delta \alpha$ and $\Delta \delta$
3. it assumes an absolute center at $(\alpha,\,\delta)$

In [9]:
vvec = R.T @ Matrix([[vra],[vdec],[vr]])
delta_v_sphere = diff(R, ra)*vvec*delta_ra + diff(R, dec)*vvec*delta_dec

In [10]:
simplify(delta_v_sphere)

Matrix([
[     \Delta\alpha*(v_\delta*sin(delta) - v_r*cos(delta))],
[    -\Delta\alpha*v_\alpha*sin(delta) - \Delta\delta*v_r],
[\Delta\alpha*v_\alpha*cos(delta) + \Delta\delta*v_\delta]])

In [11]:
print(latex(simplify(delta_v_sphere)))

\left[\begin{matrix}\Delta\alpha \left(v_\delta \sin{\left(\delta \right)} - v_{r} \cos{\left(\delta \right)}\right)\\- \Delta\alpha v_\alpha \sin{\left(\delta \right)} - \Delta\delta v_{r}\\\Delta\alpha v_\alpha \cos{\left(\delta \right)} + \Delta\delta v_\delta\end{matrix}\right]


## A special case: $\vec{v}_0$ is radial: perspective expansion/contraction

When $\vec{v}_0$ is exactly radial at $(\alpha,\,\delta)$:

In [12]:
v_radial = Matrix([
    [0],
    [0],
    [vr]
])

v0 = R.T * v_radial

In [13]:
dMdrav0 = simplify(diff(R, ra) * v0)
dMddecv0 = simplify(diff(R, dec)*v0)

dMdrav0*delta_ra + dMddecv0*delta_dec

Matrix([
[-\Delta\alpha*v_r*cos(delta)],
[           -\Delta\delta*v_r],
[                           0]])

$$ \left[\begin{matrix} \Delta v_\alpha \\ \Delta v_\delta \end{matrix} \right] =
- \left[\begin{matrix} \cos\delta & 0 \\ 0 & 1 \end{matrix}\right] v_r
\left[ \begin{matrix} \Delta \alpha \\ \Delta \delta \end{matrix} \right] $$

Since $\cos\delta>0$ always, and noting that there is not cross-term, this means that the signs of projected velocity gradient $\delta v_\alpha$ and $\delta v_\delta$ depends only on the sign of $v_r$: when $v_r>0$ (receding), the projected velocities decrease outward, i.e., we see an apparent contraction and vice versa.

## Second-order terms

One can expand to second-order as well. There will always be a higher-order correction as $\sin$ and $\cos$ expand forever.
The next order term will dominate the residual pattern.

In [14]:
delta_v_sphere2 = simplify(diff(R, ra, 2) *v0 * delta_ra**2  + diff(R, dec, 2) *v0 * delta_dec**2 + 2*diff(R, ra,dec) * v0 * delta_ra * delta_dec)

In [15]:
delta_v_sphere2.subs({ra:np.deg2rad(45), dec:np.deg2rad(45)})

Matrix([
[                                           0],
[                     0.5*\Delta\alpha**2*v_r],
[-v_r*(0.5*\Delta\alpha**2 + \Delta\delta**2)]])

In [16]:
delta_v_sphere2.subs({ra:np.deg2rad(135), dec:np.deg2rad(135)})

Matrix([
[                                           0],
[                    -0.5*\Delta\alpha**2*v_r],
[-v_r*(0.5*\Delta\alpha**2 + \Delta\delta**2)]])

In [21]:
N(delta_v_sphere.subs({ra:np.deg2rad(45),dec:np.deg2rad(45), vx:5.0, vy:5.0, vz:7.07106781}), 3)

Matrix([
[              \Delta\alpha*(0.707*v_\delta - 0.707*v_r)],
[    -0.707*\Delta\alpha*v_\alpha - 1.0*\Delta\delta*v_r],
[0.707*\Delta\alpha*v_\alpha + 1.0*\Delta\delta*v_\delta]])

In [22]:
pos2 = N(delta_v_sphere.subs({ra:np.deg2rad(300),dec:np.deg2rad(45), vx:5.0, vy:5.0, vz:7.07106781}), 3, )

In [39]:
N(delta_v_sphere.subs({ra:np.deg2rad(340),dec:np.deg2rad(-65), vx:5.0, vy:5.0, vz:7.07106781}), 3)

Matrix([
[                   -2.99*\Delta\alpha],
[5.81*\Delta\alpha + 5.15*\Delta\delta],
[ 2.71*\Delta\alpha + 5.7*\Delta\delta]])